In [1]:
import pandas as pd
import gzip
import json
from pathlib import Path
import tqdm

raf_path = Path('../2-9.playwright/output/raf/')
imr = Path('./imr')

imr.mkdir(parents=True, exist_ok=True)

def deal(path):
    with gzip.open(path, 'rt') as fp:
        ret = json.load(fp)
    ret['raf_path'] = path.as_posix()
    idx = path.as_posix().split('/')[-1].split('.')[0]
    try:
        ret['idx'] = int(idx)
    except ValueError:
        ret['idx'] = idx
    return ret

In [2]:
# 0413 + 0501_czy = 0501-df_label
df_label = pd.read_pickle('./output/0501-df_label-raf.pkl.zst.bak', compression='zstd')
df_label

,url,category,label_ERROR,label_usecase,label_interactions,label_comments
0,http://0ldsk00l.ca/,['bq'],0,1,0,
1,http://1000.chromeexperiments.com,[awwwards],0,1,1,
2,http://1000000-lives.com/,['bq'],0,3,0,
3,http://10k.global/,['bq'],0,1,1,
4,http://123screenprotector.com/,['bq'],0,3,0,
...,...,...,...,...,...,...
29237,https://map.e-mangistau.kz/,['bq'],1,<NA>,<NA>,<NA>
29238,https://map.e-zhetisu.kz/,['bq'],0,1,1,
29239,https://map.iturkistan.kz/,['bq'],1,<NA>,<NA>,<NA>
29240,https://qrg.geoportal.kz/,['bq'],1,<NA>,<NA>,<NA>


In [3]:
if Path('imr/0501-df_raw_all-raf.pkl.zst').exists():
    df = pd.read_pickle('imr/0501-df_raw_all-raf.pkl.zst', compression='zstd')
else:
    df = pd.DataFrame([deal(p) for p in tqdm.tqdm(sorted(raf_path.glob("*.gz")))]).set_index('idx')
    df = df_label.drop(columns=['url']).join(df)
    df.to_pickle('imr/0501-df_raw_all-raf.pkl.zst', compression='zstd')

100%|██████████| 29242/29242 [05:50<00:00, 83.36it/s] 


In [4]:
if Path('imr/0501-df_frame-raf.pkl.zst').exists():
    df_frame = pd.read_pickle('imr/0501-df_frame-raf.pkl.zst', compression='zstd')
else:
    df.dropna(subset=['frame'], inplace=True)
    df_frame = df.drop(columns=['frame', 'events_time_hp']).join(
        pd.DataFrame(df['frame'].to_list(), index=df.index)['gl_rafs'].explode()).reset_index(drop=False).rename(columns={'index': 'idx'})
    df_frame = df_frame.drop(columns=['gl_rafs']).join(
        pd.DataFrame(df_frame['gl_rafs'].str['value'].to_list(), index=df_frame.index)['data'])
    df_frame = df_frame.drop(columns=['data']).join(pd.json_normalize(df_frame['data']).add_prefix('frame.'), how='right')
    df_frame.dropna(subset=['frame.counter.usedWebGL'], inplace=True)
    df_frame.to_pickle('imr/0501-df_frame-raf.pkl.zst', compression='zstd')

In [5]:
df_frame[df_frame.idx==578]

,idx,category,label_ERROR,label_usecase,label_interactions,label_comments,url,date,netIdleTimeout,raf_path,frame.url,frame.title,frame.date,frame.when,frame.contextInfo,frame.counter.frames,frame.counter.contextsNum,frame.counter.usedWebGL,frame.counter.usedOffScreenCanvas
2020,578,['bq'],0,1,1,,http://dinohunt2.ivank.net/,1683100415549,1,../2-9.playwright/output/raf/00578.json.gz,https://dinohunt2.ivank.net/app5/index.html,,1683100460609,43161.2,"[{'extensions': [], 'programInfo': [{'useCount...",2149.0,1.0,9212.1,False


In [6]:
if Path('imr/0501-df_context-raf.pkl.zst').exists():
    df_context = pd.read_pickle('imr/0501-df_context-raf.pkl.zst', compression='zstd')
else:
    df_context = df_frame.explode('frame.contextInfo').reset_index(drop=False).rename(columns={'index': 'idx-frame'})
    df_context = df_context.drop(columns=['frame.contextInfo']).join(
        pd.json_normalize(df_context['frame.contextInfo']).add_prefix('frame.context.'))
    df_context.to_pickle('imr/0501-df_context-raf.pkl.zst', compression='zstd')

In [7]:
if Path('imr/0501-df_context-noraf.pkl.zst').exists():
    df_context_noraf = pd.read_pickle('imr/0501-df_context-noraf.pkl.zst', compression='zstd')
else:
    df_context_noraf = df_context.drop(columns=['frame.context.maghsk.rafList'])
    df_context_noraf.to_pickle('imr/0501-df_context-noraf.pkl.zst', compression='zstd')

In [8]:
if Path('imr/0501-df_shader-noraf.pkl.zst').exists():
    df_shader = pd.read_pickle('imr/0501-df_shader-noraf.pkl.zst', compression='zstd')
else:
    df_program = df_context_noraf[['idx-frame','idx','category','label_ERROR','label_usecase','label_interactions','label_comments','url','date','raf_path','frame.url','frame.title','frame.date','frame.when','frame.context.programInfo',]]
    # df_program['fframe.context.programInfo'] = df_program['frame.context.programInfo'].apply(lambda x: x[1:])
    df_program = df_program.explode('frame.context.programInfo').reset_index(drop=False).rename(columns={'index': 'idx-context'})
    df_program = df_program.drop(columns='frame.context.programInfo').join(pd.json_normalize(df_program['frame.context.programInfo']).add_prefix('frame.context.program.'))
    df_shader = df_program.explode('frame.context.program.vertexShaders').explode('frame.context.program.fragmentShaders')
    tmp = pd.melt(
        df_shader,
        ignore_index=False,
        value_vars=['frame.context.program.vertexShaders', 'frame.context.program.fragmentShaders'],
        var_name='frame.context.program.shader.type',
        value_name='frame.context.program.shader.source'
        ).replace({
            'frame.context.program.vertexShaders': 'vertex',
            'frame.context.program.fragmentShaders': 'fragment'
        })
    df_shader = df_shader.drop(columns=['frame.context.program.vertexShaders', 'frame.context.program.fragmentShaders']).join(tmp).dropna(subset=['frame.context.program.shader.source']).reset_index(drop=False).rename(columns={'index': 'idx-program'})
    df_shader.to_pickle('imr/0501-df_shader-noraf.pkl.zst', compression='zstd')